I understood tk as python's tkinter at first time, and made a code to represent the data as a GUI. 

In [ ]:
# Data Visualization With tkinter
import tkinter
import tkinter.ttk
import pandas as pd


file_path = '../Data/subway_crd_line_info-main/Cordinate_Subway.csv'
df = pd.read_csv(file_path, encoding='cp949')


root = tkinter.Tk()
root.title("Subway of Seoul")
root.geometry("800x600+100+100")
root.resizable(False, False)

lbl = tkinter.Label(root, text="Subway of Seoul")
lbl.pack()


treeview = tkinter.ttk.Treeview(root, columns=["one", "two", "three", "four", "five"], displaycolumns=["one", "two", "three", "four", "five"])
treeview.pack(expand=True, fill='both')


treeview.column("#0", width=50)
treeview.heading("#0", text="Index")

treeview.column("one", width=100, anchor="center")
treeview.heading("one", text="Subway_code", anchor="center")

treeview.column("two", width=150, anchor="center")
treeview.heading("two", text="Subway_name", anchor="center")

treeview.column("three", width=200, anchor="center")
treeview.heading("three", text="Subway_Engname)", anchor="center")

treeview.column("four", width=100, anchor="center")
treeview.heading("four", text="Line", anchor="center")

treeview.column("five", width=100, anchor="center")
treeview.heading("five", text="Exterior_Code", anchor="center")


for i, row in df.iterrows():
    treeview.insert('', 'end', text=i, values=(row['Subway_code'], row['Subway_name'], row['Subway_Engname'], row['Line'], row['Exterior_Code']), iid=str(i) + ".")


root.mainloop()

Two dataset for Subway in Korea.
1. Coordinate_subway = ../Data/subway_crd_line_info-main/Cordinate_Subway.csv
2. Information_subway_by_Line = ../Data/subway_crd_line_info-main/nformation_Subway_by_Line.csv

In [2]:
import pandas as pd
import os 
from geopandas import GeoDataFrame
import pickle
from shapely.geometry import Point

cur_path = os.getcwd() 

subway_crd = pd.read_csv('../Data/subway_crd_line_info-main/Cordinate_Subway.csv', encoding='cp949')
subway_line= pd.read_csv('../Data/subway_crd_line_info-main/nformation_Subway_by_Line.csv', encoding='cp949')

line_ls = []
for i in range(len(subway_crd)): 
    cur_subway = subway_crd.iloc[i]['역이름']
    matched_line = subway_line.loc[subway_line['Subway_name'] == cur_subway]
    
    if len(matched_line) > 0 : 
        line_ls.append(list(matched_line['Line']))
        
    else: 
        line_ls.append(None)

subway_crd['호선'] = line_ls # 호선 정보 부여
subway_crd.dropna(subset=['호선'], inplace=True, axis=0) # 수도권 지하철역 좌표는 노선 정보가 안들어갔으므로 제거 

subway_crd['geometry'] = subway_crd.apply(lambda x: Point(x['x'], x['y']), axis=1) # 좌표 정보를 합쳐서 geometry 정보 부여  
subway_crd_gdf = GeoDataFrame(subway_crd, crs='EPSG:4326', geometry='geometry') # 좌표계 지정

In [3]:
def write_data(data, name):
    with open(name + '.bin', 'wb') as f:
        pickle.dump(data, f)
        
        
def load_data(name):
    with open(name + '.bin', 'rb') as f:
        data = pickle.load(f)
    return data  

In [4]:
# bin 데이터 -- 파이썬 활용 용도 (ArcGIS/QGIS 시각화 용도 x)
write_data(subway_crd_gdf, 'subway_crd_4326')
subway_crd_5179 = subway_crd_gdf.to_crs(epsg=5179) # 거리 계산을 위한 좌표계로 변환 후 저장
write_data(subway_crd_5179, 'subway_crd_5179')


# 나중에 bin 데이터 다시 불러올 때 
subway_crd_4326 = load_data('subway_crd_4326')
subway_crd_5179 = load_data('subway_crd_5179')


# 노선 정보를 빼고 좌표 데이터 
subway_crd_5179 = subway_crd_5179.drop(['Line'], axis=1)
subway_crd_5179.to_file('subway_crd_5179.geojson', driver='GeoJSON')